In [18]:
from pyspark.sql import SQLContext
from pyspark.context import SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
# from notebooks import utils
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
%matplotlib inline

In [2]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [67]:
df=sqlContext.read.load('minute_weather.csv', format='com.databricks.spark.csv', header='true', inferSchema='true')
df.count()

1587257

In [68]:
filteredDF = df.filter((df.rowID % 10) == 0)
print(type(filteredDF))
filteredDF.count()

<class 'pyspark.sql.dataframe.DataFrame'>


158726

In [72]:
# filteredDF.columns
filteredDF = filteredDF.na.drop()
# filteredDF.count()
# filteredDF.columns
filteredDF.stat.corr('min_wind_speed','avg_wind_speed')

0.9708620631891071

In [6]:
filteredDF.filter(filteredDF.rain_accumulation == 0.0).count()

157812

In [7]:
filteredDF.filter(filteredDF.rain_duration == 0.0).count()

157237

In [66]:
workingDF = filteredDF.drop('rain_accumulation').drop('rain_duration').drop('hpwren_timestamp')
type(workingDF)

AttributeError: 'function' object has no attribute 'drop'

In [9]:
before = workingDF.count()
workingDF = workingDF.na.drop()
after = workingDF.count()
before - after

46

In [10]:
workingDF.columns

['rowID',
 'air_pressure',
 'air_temp',
 'avg_wind_direction',
 'avg_wind_speed',
 'max_wind_direction',
 'max_wind_speed',
 'min_wind_direction',
 'min_wind_speed',
 'relative_humidity']

In [13]:
featuresUsed = ['air_pressure',
 'air_temp',
 'avg_wind_direction',
 'avg_wind_speed',
 'max_wind_direction',
 'max_wind_speed',
 'relative_humidity']

assembler = VectorAssembler(inputCols = featuresUsed, outputCol = 'features_unscaled')
assembled = assembler.transform(workingDF)

In [34]:
scaler = StandardScaler(inputCol='features_unscaled',outputCol='features',withStd=True, withMean=True)
scalerModel = scaler.fit(assembled)
scaledData = scalerModel.transform(assembled)
newdf = scaledData.toPandas()

In [35]:
scaledData = scaledData.select('features','rowID')
elbowSet = scaledData.filter((scaledData.rowID % 3)==0).select('features')
elbowSet.columns
elbowSet.persist()

DataFrame[features: vector]

In [60]:
X = newdf
data = []

row = X.shape[0]
for i in range(row):
    data.append(list(np.array(X['features'][i])))

data = np.array(data)

In [62]:
X = data
K = range(2,31)
distortions = []

for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(X)
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

KeyboardInterrupt: 